Running the whole thing iteratively here cause notebooks are difficult


Installations

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! sudo apt install tesseract-ocr
! pip install Pillow
! pip install pytesseract
import pytesseract
from google.colab import files
from io import BytesIO
from PIL import Image,ImageFilter
#from google.colab import drive
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install tesseract
!pip install face_recognition
!pip install fe

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 5s (872 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../tesseract-

Imports

In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import pandas as pd
import requests
import json

from datetime import datetime
from PIL import Image
from numpy import genfromtxt

Pre steps

In [ ]:
# Let us create a file
with open('new-girl-dataset-batch4.json', 'w') as fp:
    pass

In [ ]:
# construct map from file encodings
characters_order = []
character_encodings = []
for filename in os.listdir('encodings'):
  try:
    character_encodings.append(read_from_file(filename))
    characters_order.append(filename[:-4])
  except:
    print('oops could not read encoding from ' + filename)

In [ ]:
TIMESTAMP_FORMAT = '%Y-%m-%d %I.%M.%S%p'

endtime_season1 = datetime.strptime("2021-07-08 11.07.00PM", TIMESTAMP_FORMAT)
endtime_season2 = datetime.strptime("2021-07-27 9.15.00PM", TIMESTAMP_FORMAT)

Functions

In [ ]:
# encoding one image
def face_encode(image_name):
  char_img = face_recognition.load_image_file('characters/' + image_name)
  char_encoding = face_recognition.face_encodings(char_img)[0]
  return char_encoding

In [ ]:
# saving all encodings to file and reading for later executions
def write_to_file(data, filename):
    with open('encodings/' + filename + '.csv', 'wb') as f:
        np.savetxt(f, data, delimiter=",")


def read_from_file(filename):
    data = genfromtxt('encodings/' + filename, delimiter=',')
    return data

In [ ]:
def fetch_image(filename, file_link):
  # fetch image from source
  response = requests.get(file_link)

  file = open(filename, "wb")
  file.write(response.content)
  file.close()

In [ ]:
def read_lines(data):
  # cropping image
  img = Image.open(data['filename'])
  width, height = img.size
  # cropping image, removing top half
  img_cropped = img.crop((0, height/2, width, height))

  # using tessarect to read
  pytesseract.pytesseract.tesseract_cmd = (
      r'/usr/bin/tesseract'
  )
  text = pytesseract.image_to_string(img_cropped)
  return text[:-1].strip().replace('\n', ' ').replace('|', 'I')

In [ ]:
def check_for_characters(match):
  characters = []
  for i in range(len(character_encodings)):
    if match[i]:
      characters.append(characters_order[i])
  return characters

In [ ]:
def find_characters(data):
  load_img = face_recognition.load_image_file(data['filename'])

  img_encoding = face_recognition.face_encodings(load_img)
  characters = []
  for face_encoding in img_encoding:
    if (face_encoding.any()):
      match = face_recognition.compare_faces(character_encodings, face_encoding)
      # ok make this check more efficient
      chars = check_for_characters(match)
      characters.extend(chars)
  return characters

In [ ]:
def fetch_timestamp(filename):
  begin = 10 # skipping the words screenshot
  end = -1
  if filename.find('PM') != -1:
    end = filename.find('PM')
  else:
    end = filename.find('AM')
  if end == -1:
    return
  end = end + 2
  timestamp = filename[begin:end]
  timestamp = timestamp.replace('at', ' ')
  return timestamp

In [ ]:
def map_season(timestamp):
  timestamp = datetime.strptime(timestamp, TIMESTAMP_FORMAT)
  if timestamp > endtime_season2:
    return 3
  if timestamp > endtime_season1:
    return 2
  return 1

In [ ]:
def append_to_output(data):
  with open('new-girl-dataset-batch60.json', 'w') as fp:
    json.dump(data, fp)

Iterating loop of all the different features

In [ ]:
with open('out.json', 'r') as f:
  files = json.load(f)

In [ ]:
data_list = []
for resource in files['resources']:
  filename = resource['display_name'] + '.' + resource['format']
  file_link = resource['url']
  fetch_image(filename, file_link)
  data = {}
  data['filename'] = filename
  data['imageLink'] = file_link
  subtitles = read_lines(data)
  if subtitles:
    data['subtitle'] = subtitles
  else:
    data['subtitle'] = 'NA'
  data['characters'] = find_characters(data)
  data['timestamp'] = fetch_timestamp(data['filename'])
  data['season'] = map_season(data['timestamp'])
  data_list.append(data)
append_to_output(data_list)